# Salary survey: Data analysis for salary among managers in the world. 

This will be a project based on the CRISP model, this means here will be developed six steps for the analysis:

1.- Business understanding.

2.- Data understanding.

3.- Preparation.

4.- Modeling.

5.- Evaluation.

6.- Deploy.

Maybe here can be used NLP for open questions. 

Will be developed a story about a fictional organization using this data. 


Data recovered from here: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

In [1]:
# Imports cell
import pandas as pd
import re 

In [2]:
salary_survey_data:pd.DataFrame = pd.read_csv('./data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1 (1).csv', sep=',')

column_names = salary_survey_data.columns


In [3]:
salary_survey_data[salary_survey_data.columns[5]]

0        55,000
1        54,600
2        34,000
3        62,000
4        60,000
          ...  
27917     78000
27918     54000
27919     89000
27920     60000
27921     62400
Name: What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.), Length: 27922, dtype: object

In [3]:
salary_survey_data.head()

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [4]:

# salary_survey_data.columns.values[5] = re.sub("[\(\[].*?[\)\]]", "", column_names[5])

# salary_survey_data.columns.values[6] = re.sub("[\(\[].*?[\)\]]", "", column_names[6])


# salary_survey_data.head()

In [5]:
salary_survey_data.shape

(27922, 18)

In [6]:
column_dictionary = {}

for name_number in range(len(salary_survey_data.columns)):
  column_new_name = f'column{name_number + 1}'
  column_dictionary[column_new_name] = salary_survey_data.columns[name_number]
  salary_survey_data.columns.values[name_number] = column_new_name

print(salary_survey_data.columns)



Index(['column1', 'column2', 'column3', 'column4', 'column5', 'column6',
       'column7', 'column8', 'column9', 'column10', 'column11', 'column12',
       'column13', 'column14', 'column15', 'column16', 'column17', 'column18'],
      dtype='object')


In [7]:

salary_survey_data[salary_survey_data.columns[5]]

KeyError: 'column6'